# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Your code here
df = pd.read_csv('homepage_actions.csv')
df.head(10)

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
5,2016-09-24 20:32:25.712659,681598,experiment,view
6,2016-09-24 20:39:03.248853,522116,experiment,view
7,2016-09-24 20:57:20.336757,349125,experiment,view
8,2016-09-24 20:58:01.948663,349125,experiment,click
9,2016-09-24 21:00:12.278374,560027,control,view


In [3]:
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [4]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


* Start investigating the id column: 
    - `id` is not unique. Upon verifying, people who `click` are also the ones that `view`.  
    
* How many viewers also clicked?
    - Within the `control` group, approximately 27.97% viewers also clicked
    - Within the `experiment` group, approximately 30.97% viewers also clicked
    
    
* Are there any anomalies with the data; did anyone click who didn't view? 
    - Haven't found any anomalies so far
    
* Is there any overlap between the control and experiment groups? * If so, how do you plan to account for this in your experimental design?
    - The `id` list of those who `click` is completely within the `id` list of people who `view`

In [6]:
# Separate the 2 groups into their own dataframe
experiment = df[df.group == 'experiment']
control = df[df.group == 'control']

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

1. Metrics to be studied: `len(click) / len(view)` ratio between the control and expermiment groups


2. Define Hypotheses:
- Null Hypotheses: There is no significant difference in the click-through rates between the control and experiment groups
    `H_0: muE = muC`
    
- Alternative Hypotheses: There is a difference in the click-through rates between the control and experiment groups
    `H_A: muE ≠ muC`

In [16]:
# Add another column called 'click' to control & experiment df
control['click'] = control.action.apply(lambda x: 1 if x == 'click' else 0)
experiment['click'] = experiment.action.apply(lambda x: 1 if x == 'click' else 0)

# Convert them to Series with 'id' as index and 1/0 as click? (1 = clicked, 0 = not clicked)
control_clicks = control.groupby('id').click.sum()
experiment_clicks = experiment.groupby('id').click.sum()

control_clicks.head()

id
182994    1
183089    0
183248    1
183515    0
183524    0
Name: click, dtype: int64

In [17]:
experiment_clicks.head()

id
182988    0
183136    0
183141    1
183283    0
183389    0
Name: click, dtype: int64

In [47]:
# Print out some summary results
print('\t\tSize \tClick Count \tClick Rate')
print(f'Control \t{len(control_clicks)} \t{control_clicks.sum()} \t\t{control_clicks.mean()}')
print(f'Experiment \t{len(experiment_clicks)} \t{experiment_clicks.sum()} \t\t{experiment_clicks.mean()}')

		Size 	Click Count 	Click Rate
Control 	3332 	932 		0.2797118847539016
Experiment 	2996 	928 		0.3097463284379172


In [12]:
# Applying 'ttest_ind()' with equal_var = False (Welch's Test)
stats.ttest_ind(control_clicks, experiment_clicks, equal_var=False).pvalue/2

0.004466402814337101

p-value = 0.004466402814337101 < 0.05 --> We can **Reject the Null Hypothesis**

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [22]:
# control group's click rate
rC = control_clicks.mean()

In [33]:
# experiment's click count if it has the same rate as the control group
estimated_E = len(experiment_clicks) * rC

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [42]:
# Standard Deviation for binomial variable in experiment group if it has the same click rate as control's
n = len(experiment_clicks)
p = rC
std = (n * p * (1-p))**.5
std

24.568547907005815

In [43]:
# Calculate z-score = (actual experiment click - estimated experiment click) / std
z_score = (experiment_clicks.sum() - estimated_E) / std
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [46]:
# Your code here
p_value = 1 - stats.norm.cdf(z_score)
p_value

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Both methods result with p-values < 0.05 --> **Rejection of the Null Hypothesis**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.